In [1]:
%load_ext autoreload
%autoreload 2 

In [1]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/work/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
import os

# from test_transformers.src.test2_transformers.models.t5.modeling_t5 import T5ForConditionalGeneration
# from test_transformers.src.test2_transformers.models.longt5.modeling_longt5 import LongT5ForConditionalGeneration
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1, 2"
import argparse
import logging
import time
import torch


In [4]:
from data import data_process
from inference import keyphrases_selection,calculate_score, get_score_df
from torch.utils.data import DataLoader
# from transformers import T5ForConditionalGeneration, LongT5ForConditionalGeneration


def get_setting_dict():
    setting_dict = {}
    setting_dict["max_len"] = 512
    
    # setting_dict["temp_en1"] = "Answer the following question by reasoning step-by-step. "
    setting_dict["temp_en2"] = "Keyword extraction: Extract keywords that reflect the key ideas of this book. "

    setting_dict["temp_en1"] = "Book:"
    setting_dict["temp_de1"] = "This book mainly talks about "
    # setting_dict["temp_de1"] = "Keyphrases: "
    setting_dict["temp_de2"] = "And in certain sections, this book talks about "
    setting_dict["temp_de3"] = "Therefore, the keyphrases of this text are "
    setting_dict["model"] = "base"
    setting_dict["enable_filter"] = False
    setting_dict["enable_pos"] = True
    setting_dict["enable_freq"] = False
    setting_dict["enable_att"] = True
    setting_dict["position_factor"] = 1.2e8
    setting_dict["length_factor"] = 1.2
    return setting_dict

def parse_argument(dataset_dir=None, dataset_name=None, batch_size=None, log_dir=None):

    if not all([dataset_dir, dataset_name, batch_size, log_dir]):
        raise ValueError("All arguments (dataset_dir, dataset_name, batch_size, log_dir) must be provided.")

    import argparse
    parser = argparse.Namespace(
        dataset_dir=dataset_dir,
        dataset_name=dataset_name,
        batch_size=batch_size,
        log_dir=log_dir
    )
    return parser





In [5]:
dataset = "DUC2001"
args = parse_argument(
    dataset_dir=f"data/{dataset}", 
    dataset_name=f"{dataset}",
    batch_size=12,
    log_dir="path/to/log"
)

# 실험하자자자

In [7]:
from transformers import GemmaTokenizer
from transformers import Gemma2ForCausalLM
# from test_transformers.src.test2_transformers.models.gemma2.modeling_gemma2_normal import Gemma2ForCausalLM
from transformers import BitsAndBytesConfig
setting_dict = get_setting_dict()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = GemmaTokenizer.from_pretrained("google/gemma-2-9b", padding_side = 'left')
# tokenizer = GemmaTokenizer.from_pretrained(
#     "google/gemma-2-9b-it",
#     padding_side="left",  # 생성 작업 필수
#     truncation_side="left",
#     pad_token="<pad>",  # 명시적 패딩 토큰 지정
#     add_eos_token=True  # 문장 종료 신호 강화
# )


# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.bfloat16  # 계산 속도 향상
# )
quant_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = Gemma2ForCausalLM.from_pretrained(
    "google/gemma-2-9b",
    quantization_config=quant_config,
    # device_map="auto",
    device_map={"": 0},
    torch_dtype=torch.bfloat16,
    output_attentions=True 
)

dataset, doc_list, labels, labels_stemed, docs_candidates, att_candidates, cand_sim_group = data_process(setting_dict, args.dataset_dir, args.dataset_name, 23,1, model, device)
dataloader = DataLoader(dataset, num_workers=12, batch_size=args.batch_size)

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

...

/home/work/.default/anaconda3/envs/hyeongu_base/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_attentions` is. When `return_dict_in_generate` is not `True`, `output_attentions` is ignored.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

The 'batch_size' argument of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.
`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


candidate_num:  35926
examples:  35829


In [7]:
del model
torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [8]:
from test_transformers.src.test2_transformers.models.gemma2.modeling_gemma2_normal import Gemma2ForCausalLM_our

model2 = Gemma2ForCausalLM_our.from_pretrained(
    "google/gemma-2-9b",
    quantization_config=quant_config,
#     device_map="auto",
    device_map={"":0},
    torch_dtype=torch.bfloat16,
    output_attentions=True 
)

This is the weighted gemma2 version!


/home/work/.default/hyeongu/PromptRank_cross_att_gemma2/test_transformers/src/test2_transformers/generation/configuration_utils.py:816: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_attentions` is. When `return_dict_in_generate` is not `True`, `output_attentions` is ignored.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
cosine_similarity_rank, pred_labels = keyphrases_selection(setting_dict, doc_list, labels_stemed, labels, model2, dataloader,0.3,0.4,15,1, device)
# cosine_similarity_rank, ori_pred_labels =keyphrases_selection_original(setting_dict, doc_list, labels_stemed, labels,  model, dataloader, device)

[autoreload of transformers failed: Traceback (most recent call last):
  File "/home/work/.default/anaconda3/envs/hyeongu_base/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/work/.default/anaconda3/envs/hyeongu_base/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/work/.default/anaconda3/envs/hyeongu_base/lib/python3.10/importlib/__init__.py", line 139, in reload
    name = module.__spec__.name
  File "/home/work/.default/anaconda3/envs/hyeongu_base/lib/python3.10/site-packages/transformers/utils/import_utils.py", line 1789, in __getattr__
    if name in self._objects:
  File "/home/work/.default/anaconda3/envs/hyeongu_base/lib/python3.10/site-packages/transformers/utils/import_utils.py", line 1789, in __getattr__
    if name in self._objects:
  File "/home/work/.default/anaconda3/envs/hyeongu_base/lib/python3.1

OutOfMemoryError: CUDA out of memory. Tried to allocate 448.00 MiB. GPU 0 has a total capacity of 79.19 GiB of which 401.94 MiB is free. Process 2245 has 8.00 GiB memory in use. Process 791524 has 18.04 GiB memory in use. Including non-PyTorch memory, this process has 52.74 GiB memory in use. Of the allocated memory 50.54 GiB is allocated by PyTorch, and 1.53 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [13]:
a,df2=calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,0,1,1,0,0.7,1.2e8,0.6,2)

0.3281872509960159 0.3696357735304724 0.3594104308390023


In [ ]:
a,df2=calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,0,1,1,0,0.7,1.2e8,0.6,0)

In [14]:
a

{'f5': 32.82, 'f10': 36.96, 'f15': 35.94}

In [15]:
df2.to_csv('duc_our_out.csv', index=False)

In [13]:
df[df['doc_id']==125][:20]

,doc_id,candidate,score,pos,att_score,candidate_len,whole_att_score,label
11245,tensor(125),abimael guzman,tensor(-2.9557),tensor(75),"tensor(0.5713, dtype=torch.float64)",tensor(5),"tensor(0.9258, dtype=torch.float64)",0
11246,tensor(125),10-point agreement,tensor(-3.4281),tensor(138),"tensor(0.5757, dtype=torch.float64)",tensor(6),"tensor(0.6602, dtype=torch.float64)",1
11247,tensor(125),peace agreement,tensor(-3.4686),tensor(44),"tensor(1.1904, dtype=torch.float64)",tensor(2),"tensor(1.1250, dtype=torch.float64)",1
11248,tensor(125),president alberto fujimori,tensor(-3.7336),tensor(65),"tensor(1.0125, dtype=torch.float64)",tensor(5),"tensor(0.7930, dtype=torch.float64)",0
11249,tensor(125),terrorism,tensor(-4.2999),tensor(158),"tensor(1.2139, dtype=torch.float64)",tensor(1),"tensor(1.5547, dtype=torch.float64)",0
11250,tensor(125),war,tensor(-4.5909),tensor(237),"tensor(1.0017, dtype=torch.float64)",tensor(1),"tensor(0.7266, dtype=torch.float64)",0
11251,tensor(125),peruvian government,tensor(-4.6402),tensor(59),"tensor(0.8962, dtype=torch.float64)",tensor(3),"tensor(1.1250, dtype=torch.float64)",1
11252,tensor(125),popular war,tensor(-5.0873),tensor(149),"tensor(0.8358, dtype=torch.float64)",tensor(2),"tensor(1.4453, dtype=torch.float64)",1
11253,tensor(125),agreement,tensor(-5.1082),tensor(381),"tensor(0.8342, dtype=torch.float64)",tensor(1),"tensor(0.1777, dtype=torch.float64)",0
11254,tensor(125),general amnesty,tensor(-5.2777),tensor(216),"tensor(2.3247, dtype=torch.float64)",tensor(2),"tensor(1.0156, dtype=torch.float64)",1


In [10]:
a=calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,0,0.6,1.2e8,0.7,2)

19.17
23.06
22.89


In [11]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)

(3, 384)


In [11]:
initial = [0.]
scaling=[0.6,0.7]
print("att_layer")
for i in range(1):
    for j in range(2):
        print(initial[i], scaling[j])
        print('local')
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,0,1,1,initial[i],scaling[j],1.2e8,0.6,0)
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,initial[i],scaling[j],1.2e8,0.6,0)
        print('global')
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,0,1,1,initial[i],scaling[j],1.2e8,0.6,1)
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,initial[i],scaling[j],1.2e8,0.6,1)
        print('local+global')
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,0,1,1,initial[i],scaling[j],1.2e8,0.6,2)
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,initial[i],scaling[j],1.2e8,0.6,2)

att_layer
0.0 0.6
local
21.39
24.20
23.57
20.88
24.15
23.85
global
21.10
24.33
24.39
19.40
22.88
22.50
local+global
20.93
24.50
24.17
19.40
23.36
22.86
0.0 0.7
local
21.62
24.20
23.46
20.65
24.28
23.96
global
20.88
24.33
24.14
19.45
22.45
22.29
local+global
20.93
24.50
23.92
19.11
23.23
22.68


In [8]:
initial = [0.]
scaling=[0.6,0.7]
print("att_layer")
for i in range(1):
    for j in range(2):
        print(initial[i], scaling[j])
        print('local')
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,0,1,1,initial[i],scaling[j],1.2e8,0.6,0)
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,initial[i],scaling[j],1.2e8,0.6,0)
        print('global')
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,0,1,1,initial[i],scaling[j],1.2e8,0.6,1)
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,initial[i],scaling[j],1.2e8,0.6,1)
        print('local+global')
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,0,1,1,initial[i],scaling[j],1.2e8,0.6,2)
        calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,initial[i],scaling[j],1.2e8,0.6,2)

att_layer
0.0 0.6
local
21.73
24.15
23.57
20.53
24.20
23.78
global
21.56
24.68
24.35
20.19
23.06
23.18
local+global
21.79
24.41
24.42
20.25
23.67
23.28
0.0 0.7
local
21.62
24.20
23.39
20.48
24.15
23.68
global
21.56
24.98
24.24
20.36
22.97
23.11
local+global
21.73
24.55
24.32
20.25
23.45
23.28


In [9]:
aa=calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,0.,1,1.2e8,0.,2) # +2

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


16.84
21.30
21.55


In [9]:
aa=calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,0.3,0.6,1.2e8,0.6,2)

32.57
36.64
35.54


In [13]:
aa=calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,0.,0.5,1.2e8,0.6,2)

32.52
36.35
35.54


In [14]:
aa=calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,0.,0.3,1.2e8,0.6,2)

32.17
36.35
35.69


In [34]:
ranked_keyphrases= calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,0.2,0.4,1.2e8,0.6)

18.63
24.17
23.94


In [ ]:
ranked_keyphrases= calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,0.3,0.4,1.2e8,0.6)

18.83
23.93
23.81


In [13]:
ranked_keyphrases= calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,1,1,0.6,0.6,1.2e8,0.6) #1.2

31.13
35.02
34.44


In [1]:
ranked_keyphrases= calculate_score(setting_dict, cosine_similarity_rank,doc_list,labels,labels_stemed,1,0,1,0.6,0.4,1.2e8,0.6)

NameError: name 'calculate_score' is not defined

In [7]:
ranked_keyphrases= calculate_score(setting_dict, ori_cosine_similarity_rank,doc_list,labels,labels_stemed,0,0,1,0.6,0.4,1.2e8,0.6)

0.20029895366218237
0.24172317510969288
0.23611572996341867


In [8]:
ranked_keyphrases= calculate_score(setting_dict, ori_cosine_similarity_rank,doc_list,labels,labels_stemed,1,0,1,0.6,0.4,1.2e8,0.6)

0.1993024414549078
0.240127642600718
0.23811107416029265


In [11]:
ranked_keyphrases= calculate_score(setting_dict, ori_cosine_similarity_rank,doc_list,labels,labels_stemed,0,0,1,0.6,0.4,1.2e8,0.6)

0.1993024414549078
0.24092540885520544
0.23611572996341867


In [12]:
ranked_keyphrases= calculate_score(setting_dict, ori_cosine_similarity_rank,doc_list,labels,labels_stemed,1,0,1,0.6,0.4,1.2e8,0.6)

0.1983059292476333
0.24092540885520544
0.23877618889258398


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def generate_doc_pairs(doc, candidates, idx):
    count = 0
    doc_pairs = []
    # task_prompt = "Extract keyphrases from the text. The answer should be listed after 'Keyphrase: '."
    task_prompt = "Extract keyphrases from the text. The answer should be listed after 'This text mainly talks about '."
    doc_prompt = f"Text: {doc}"
   
    # prompt = task_prompt + " " + doc_prompt
    prompt = doc_prompt
    for id, can_and_pos in enumerate(candidates):
        candidate = can_and_pos
        target_temp = "Answer: This text mainly talks about "
        target = f"Answer: This text mainly talks about {candidate}."
        full_input = prompt + " " + target 
        input_ids = tokenizer(full_input, max_length=782, padding="max_length", truncation=True, return_tensors="pt")['input_ids']
        reconstructed_text = tokenizer.convert_ids_to_tokens(input_ids[0])
        print(reconstructed_text)
        target_temp_ids = tokenizer(target_temp, return_tensors="pt")['input_ids'][0]
        target_temp_len = target_temp_ids.size(0)
        target_ids = tokenizer(target, return_tensors="pt")['input_ids'][0]
        target_len = target_ids.size(0)  # 맨 앞 '2' 토큰도 포함함
        non_pad_len = (input_ids != tokenizer.pad_token_id).sum().item() 
        context_len = non_pad_len - target_len            # 딱 Answer 토큰 앞까지만 idx 44
        start_idx = context_len + target_temp_len -1          # 딱 about 위치치     근데 inference에서 -1 더 해서 talks 위치로 했어
        candidate_len= target_len - target_temp_len
        

        print(target_len)
        # Find where the candidate tokens appear in the document
#        en_input_id_list = en_input_ids[0].tolist()
        candidate_position = None


        dic = {"candidate":candidate, "idx":target_len, "start_idx":start_idx, 'cand_len':candidate_len, "context_len":context_len} 
        
        doc_pairs.append([input_ids, dic])
        # print(tokenizer.decode(en_input_ids[0]))
        # print(tokenizer.decode(de_input_ids[0]))
        # print(candidate)
        # print(de_input_len)
        # print()
        # exit(0)

    return doc_pairs, count, target_ids, start_idx

In [ ]:
import torch.nn.functional as F
from collections import defaultdict

with torch.no_grad():
    output = model(input_ids=input_ids, attention_mask=input_mask)
    logits = output.logits  # (B, L, V)
    log_probs = F.log_softmax(logits, dim=-1)

    score = torch.zeros(input_ids.size(0))

    # 1. start_idx로 그룹 묶기
    group_by_start = defaultdict(list)
    for j in range(input_ids.size(0)):
        group_by_start[dic["start_idx"][j]].append(j)

    for start, indices in group_by_start.items():
        for j in indices:
            cand_len = dic["candidate_len"][j]
            # 예측 위치: start ~ start + cand_len - 1
            pos_indices = torch.arange(start, start + cand_len)
            token_ids = input_ids[j, pos_indices + 1]  # 다음 위치에 등장해야 할 정답 토큰

            log_probs_j = log_probs[j, pos_indices, :]  # (cand_len, V)
            token_scores = log_probs_j.gather(1, token_ids.unsqueeze(1)).squeeze(1)  # (cand_len,)

            score_j = token_scores.sum() / (cand_len ** length_factor)
            score[j] = score_j

tensor(0.9231)

In [ ]:
from collections import defaultdict
import torch.nn.functional as F

with torch.no_grad():
    output = model(input_ids=input_ids, attention_mask=input_mask)
    logits = output.logits  # (B, L, V)
    log_probs = F.log_softmax(logits, dim=-1)

    score = torch.zeros(input_ids.size(0), device=input_ids.device)

    # start_idx 기준으로 묶기
    grouped = defaultdict(list)
    for j in range(input_ids.size(0)):
        grouped[dic["start_idx"][j]].append(j)

    for start, batch_indices in grouped.items():
        for j in batch_indices:
            cand_len = dic["candidate_len"][j]
            pos = torch.arange(start, start + cand_len, device=input_ids.device)
            # 정답 토큰 ID (next token)
            token_ids = input_ids[j, pos + 1]
            log_p = log_probs[j, pos, :]  # shape: (cand_len, vocab)
            selected_log_p = log_p.gather(1, token_ids.unsqueeze(1)).squeeze(1)  # shape: (cand_len,)
            score[j] = selected_log_p.sum() / (cand_len ** length_factor)

In [8]:
calculate_score(setting_dict, ori_cosine_similarity_rank,doc_list,labels,labels_stemed,1,0,0,0.6,0.4)

0.028898854010961633
0.04946150777822098
0.07781842367808448


In [ ]:
a=get_score_df(setting_dict,cosine_similarity_rank,doc_list,labels,labels_stemed,0,0)

In [8]:
len(att_candidates[20])

74

In [ ]:
a.to_csv("result/wikihow_result_weight_0.5_0.5_not_pos.csv", index=False) 

In [ ]:
import pickle
dataset='inspec'
file_path1 = f'{dataset}_pred_labels_ori.pkl'

with open(file_path1, 'rb') as file:
    ori = pickle.load(file)
    
file_path2 = f'{dataset}_pred_labels_1_0.5.pkl'


with open(file_path2, 'rb') as file:
    cross = pickle.load(file)
    
differences = {}

for idx in ori:
    ori_temp = set(ori[idx]['temp'])
    cross_temp = set(cross[idx]['temp'])
    
    # 차이점 계산
    diff_ori = ori_temp - cross_temp  # ori에서만 있는 값
    diff_cross = cross_temp - ori_temp  # cross에서만 있는 값
    
    differences[idx] = {'ori_only': list(diff_ori), 'cross_only': list(diff_cross)}
